In [1]:
# to import .py module
import sys
import os
ROOT_DIR = os.path.abspath(os.curdir)[:-11]
sys.path.append(os.path.abspath(ROOT_DIR))

In [2]:
import codecs
import json
import Preprocess
from Parameters import *
import numpy as np
import keras
from keras.models import Sequential, Model, load_model, save_model
from keras.layers import Embedding, Input, Dense, Reshape, Concatenate, Flatten, Bidirectional, LSTM, Activation, Dropout
from keras.optimizers import Adagrad, Adam, SGD, RMSprop
from keras.regularizers import l2
from time import time
import multiprocessing as mp
import sys
import math
import tensorflow as tf
import timeit
import CRF

### Bert model

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import json

def get_model(model_url, max_seq_length):
  labse_layer = hub.KerasLayer(model_url, trainable=True)

  # Define input.
  input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                         name="input_word_ids")
  input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                     name="input_mask")
  segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                      name="segment_ids")

  # LaBSE layer.
  pooled_output,  _ = labse_layer([input_word_ids, input_mask, segment_ids])

  # The embedding is l2 normalized.
  pooled_output = tf.keras.layers.Lambda(
      lambda x: tf.nn.l2_normalize(x, axis=1))(pooled_output)

  # Define model.
  return tf.keras.Model(
        inputs=[input_word_ids, input_mask, segment_ids],
        outputs=pooled_output), labse_layer

max_seq_length = 64
labse_model, labse_layer = get_model(
    model_url="https://tfhub.dev/google/LaBSE/1", max_seq_length=max_seq_length)

In [ ]:
import bert

vocab_file = labse_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = labse_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

def create_input(input_strings, tokenizer, max_seq_length):

  input_ids_all, input_mask_all, segment_ids_all = [], [], []
  for input_string in input_strings:
    # Tokenize input.
    input_tokens = ["[CLS]"] + tokenizer.tokenize(input_string) + ["[SEP]"]
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    sequence_length = min(len(input_ids), max_seq_length)

    # Padding or truncation.
    if len(input_ids) >= max_seq_length:
      input_ids = input_ids[:max_seq_length]
    else:
      input_ids = input_ids + [0] * (max_seq_length - len(input_ids))

    input_mask = [1] * sequence_length + [0] * (max_seq_length - sequence_length)

    input_ids_all.append(input_ids)
    input_mask_all.append(input_mask)
    segment_ids_all.append([0] * max_seq_length)

  return np.array(input_ids_all), np.array(input_mask_all), np.array(segment_ids_all)

def encode(input_text):
  input_ids, input_mask, segment_ids = create_input(
    input_text, tokenizer, max_seq_length)
  return labse_model([input_ids, input_mask, segment_ids])

### Model

#### Load data and define necessary parameters, functions

In [ ]:
import json
with open(TRAIN_DATA_FILE, encoding='utf8') as f:
  trainset = json.load(f)

In [ ]:
with open(file=NORM_ADDS_FILE, mode='r', encoding='utf-8') as f:
  NORM_ADDS = json.load(fp=f)

with open(file=ID2id_FILE, mode='r', encoding='utf-8') as f:
  ID2id = json.load(fp=f)

with open(file=id2ID_FILE, mode='r', encoding='utf-8') as f:
  id2ID = json.load(fp=f)

with open(file=id2norm_add_FILE, mode='r', encoding='utf-8') as f:
  id2norm_add= json.load(fp=f)

In [ ]:
norm_embeddings = np.load(NORM_EMBEDDING_FILE, allow_pickle=True)
NT_norm_embeddings = np.load(NT_NORM_EMBEDDING_FILE, allow_pickle=True)

In [ ]:
# noisy_add = trainset['data'][119]['noisy_add']

In [ ]:
# entities = CRF.detect_entity(noisy_add)
# entities

In [ ]:
entities2index = {'street': 0, 'ward': 1, 'district': 2, 'city': 3}

In [ ]:
def create_type_add_vector(noisy_add):
  entities = CRF.detect_entity(noisy_add)
  type_add_vector = np.zeros((1,4))
  for entity in entities:
    if entity == 'name':
      pass
    else:
      index = entities2index[entity]
      type_add_vector[0, index] = 1
  return type_add_vector

In [ ]:
# for a sample in trainset, get id of norm_add coresponding to noisy_add of this sample
def get_norm_id(sample):
  return list(sample['std_add'].keys())[0]

In [ ]:
# norm_embeddings.shape
dim = 772
num_of_norm = 34481
num_negative = 1023
SNN_layers = [512, 256, 128]
MLP_layers = [64, 32, 16, 8]

#### Generator

In [ ]:
def concat(v,type_add_vector):
  return np.concatenate((v, type_add_vector), axis=1)

In [ ]:
def generator(trainset, num_negative, id2norm_add, dim):
  # for i, sample in enumerate(trainset['data'][:100]):
  for i, sample in enumerate(trainset['data'][:32932]):
    batch_size = num_negative + 1
    batch_indexes = np.array([i for i in range(batch_size)])
    noisy_adds, norm_adds, labels = [], [], []
    # noisy add có thể hoa hoặc thường, đưa về chữ thường hết và loại bỏ đi punctuation
    noisy_add = preprocess.remove_punctuation(CRF.get_better_add(sample['noisy_add']).lower())

    id = get_norm_id(sample)
    ID_ = int(id2ID[id])

    type_add_vector = create_type_add_vector(sample['noisy_add'])
    noisy_add_vector = concat(np.array(encode([noisy_add])), type_add_vector)

    # kiểm tra xem noisy_add truyền vào là thường hay hoa
    if noisy_add != preprocess.remove_tone_of_text(noisy_add):
      noisy_adds.append(noisy_add_vector)
      norm_adds.append(norm_embeddings[ID_, :].reshape(1, -1))
      labels.append(1)

      # nagative sample
      for t in range(num_negative):
        j = str(np.random.randint(num_of_norm))
        while (j not in id2norm_add) or (j == id):
          j = str(np.random.randint(num_of_norm))

        ID = int(id2ID[j])
        noisy_adds.append(noisy_add_vector)
        norm_adds.append(norm_embeddings[ID, :].reshape(1, -1))
        labels.append(0)

    else:
      noisy_adds.append(noisy_add_vector)
      norm_adds.append(NT_norm_embeddings[ID_, :].reshape(1, -1))
      labels.append(1)

      # negative sample
      for t in range(num_negative):
        j = str(np.random.randint(num_of_norm))
        while (j not in id2norm_add) or (j == id):
          j = str(np.random.randint(num_of_norm))

        ID = int(id2ID[j])
        noisy_adds.append(noisy_add_vector)
        norm_adds.append(NT_norm_embeddings[ID, :].reshape(1, -1))
        labels.append(0)

    np.random.shuffle(batch_indexes)
    yield [np.array(noisy_adds).reshape(batch_size, dim)[batch_indexes], np.array(norm_adds).reshape(batch_size, dim)[batch_indexes]], np.array(labels)[batch_indexes]

#### Model

In [ ]:
def get_SNN_model(dim,
                  SNN_layers,
                  MLP_layers, 
                  type_ = None):
    num_SNN_layers = len(SNN_layers)

    noisy_input         = Input(shape=(dim,), dtype=np.float32, name='noisy_input')
    norm_input          = Input(shape=(dim,), dtype=np.float32, name='norm_input')

    if type_ == 'add':
      noisy_latent = noisy_input
      norm_latent = norm_input
      # SNN layer
      for idx in range(num_SNN_layers):
        Dense_layer = Dense(units=SNN_layers[idx]) 
        noisy_latent = Dense_layer(noisy_latent)
        norm_latent = Dense_layer(norm_latent)

        Activation_layer = keras.layers.LeakyReLU(alpha=0.1)
        noisy_latent = Activation_layer(noisy_latent)
        norm_latent = Activation_layer(norm_latent)
        # noisy_latent = keras.layers.BatchNormalization()(noisy_latent)
        # norm_latent = keras.layers.BatchNormalization()(norm_latent)
        # noisy_latent = Dropout(0.2)(noisy_latent)
        # norm_latent = Dropout(0.2)(norm_latent)

      predict_vector = keras.layers.Multiply()([noisy_latent, norm_latent]) 

      # MLP layers
      for idx in range(len(MLP_layers)):
        Dense_layer = Dense(units=MLP_layers[idx]) 
        predict_vector = Dense_layer(predict_vector)

        Activation_layer = keras.layers.LeakyReLU(alpha=0.1)
        predict_vector = Activation_layer(predict_vector)
        # predict_vector = keras.layers.BatchNormalization()(predict_vector)   
        # predict_vector = Dropout(0.2)(predict_vector)

      # Add a dense layer with a sigmoid unit to generate the similarity score
      prediction = Dense(1,activation='sigmoid')(predict_vector)
      
      # Connect the inputs with the outputs
      siamese_net = keras.models.Model(inputs=[noisy_input,norm_input],outputs=prediction)
      
      # return the model
      return siamese_net
    elif type_ == 'cosine':
      noisy_latent = noisy_input
      norm_latent = norm_input
      # SNN layer
      for idx in range(num_SNN_layers):
        Dense_layer = Dense(units=SNN_layers[idx]) 
        noisy_latent = Dense_layer(noisy_latent)
        norm_latent = Dense_layer(norm_latent)

        Activation_layer = keras.layers.LeakyReLU(alpha=0.1)
        noisy_latent = Activation_layer(noisy_latent)
        norm_latent = Activation_layer(norm_latent)

      L1_layer = keras.layers.Lambda(lambda tensors:-keras.losses.cosine_similarity(tensors[0],tensors[1], axis=-1))
      prediction = L1_layer([noisy_latent, norm_latent])
      siamese_net = keras.models.Model(inputs=[noisy_input,norm_input],outputs=prediction)
      return siamese_net
keras.backend.clear_session()

#### Train all data

In [ ]:
SNN_model = get_SNN_model(dim, SNN_layers, MLP_layers, type_='add')
SNN_model.summary()

In [ ]:
tf.keras.utils.plot_model(SNN_model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
SNN_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss=keras.losses.binary_crossentropy)

### train

In [ ]:
losses = []
start = timeit.default_timer()
for i in range(20):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1313s 40ms/step - loss: 0.0142
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_1_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_1_epoches.snn/assets


32932/32932 [==============================] - 1296s 39ms/step - loss: 0.0031
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_2_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_2_epoches.snn/assets


32932/32932 [==============================] - 1300s 39ms/step - loss: 0.0022
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_3_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_3_epoches.snn/assets


  13802/Unknown - 540s 39ms/step - loss: 0.0018

In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_3_epoches.snn')
start = timeit.default_timer()
for i in range(3, 20):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1331s 40ms/step - loss: 0.0017
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_4_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_4_epoches.snn/assets


32932/32932 [==============================] - 1323s 40ms/step - loss: 0.0014
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_5_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_5_epoches.snn/assets


32932/32932 [==============================] - 1312s 40ms/step - loss: 0.0012
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_6_epoches.snn/assets


...

32932/32932 [==============================] - 1349s 41ms/step - loss: 4.0428e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_19_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_19_epoches.snn/assets


32932/32932 [==============================] - 1373s 42ms/step - loss: 3.9011e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_20_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_20_epoches.snn/assets


Time:  22838.600827937
[0.0016764120664447546, 0.0013780668377876282, 0.0011655717389658093, 0.0010213433997705579, 0.0008999495767056942, 0.0008202471653930843, 0.0007393497508019209, 0.0006712095346301794, 0.0006215792382135987, 0.0005813484312966466, 0.0005358031485229731, 0.0005107802571728826, 0.0004779985756613314, 0.0004453118017408997, 0.00042794813634827733, 0.000404281890951097, 0.00039011071203276515]


In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_20_epoches.snn')
start = timeit.default_timer()
for i in range(20, 40):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1290s 39ms/step - loss: 3.6864e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_21_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_21_epoches.snn/assets


32932/32932 [==============================] - 1289s 39ms/step - loss: 3.5117e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_22_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_22_epoches.snn/assets


32932/32932 [==============================] - 1270s 39ms/step - loss: 3.3995e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_23_epoches.snn/assets


...

32932/32932 [==============================] - 1263s 38ms/step - loss: 2.1193e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_39_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_39_epoches.snn/assets


32932/32932 [==============================] - 1267s 38ms/step - loss: 2.0789e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_40_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_40_epoches.snn/assets


Time:  26119.832583734
[0.0003686402924358845, 0.00035116850631311536, 0.0003399450215511024, 0.00032698179711587727, 0.00031504820799455047, 0.00030356060597114265, 0.0002930214977823198, 0.0002762348158285022, 0.0002756016911007464, 0.00026509040617384017, 0.0002599711297079921, 0.0002495658409316093, 0.00024133127590175718, 0.0002417460345895961, 0.0002291613636771217, 0.00022651323524769396, 0.00022166015696711838, 0.0002215537242591381, 0.00021192904387135059, 0.0002078896650345996]


In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_40_epoches.snn')
start = timeit.default_timer()
for i in range(40, 65):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1277s 39ms/step - loss: 2.0991e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_41_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_41_epoches.snn/assets


32932/32932 [==============================] - 1287s 39ms/step - loss: 2.0212e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_42_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_42_epoches.snn/assets


32932/32932 [==============================] - 1239s 38ms/step - loss: 1.9830e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_43_epoches.snn/assets


...

32932/32932 [==============================] - 1336s 41ms/step - loss: 1.4590e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_64_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_64_epoches.snn/assets


32932/32932 [==============================] - 1344s 41ms/step - loss: 1.5035e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_65_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_65_epoches.snn/assets


Time:  32393.350629503002
[0.0002099073026329279, 0.00020212223171256483, 0.00019829509255941957, 0.00019576511112973094, 0.00018804690625984222, 0.00018599061877466738, 0.00018260277283843607, 0.00018375925719738007, 0.0001800203463062644, 0.0001745084737194702, 0.0001710340875433758, 0.0001759362203301862, 0.00016949730343185365, 0.00016810520901344717, 0.00016217170923482627, 0.0001621651608729735, 0.000168673213920556, 0.00016265996964648366, 0.0001620137190911919, 0.00015758724475745112, 0.00015784903371240944, 0.0001525010884506628, 0.00014570918574463576, 0.00014589579950552434, 0.00015035265823826194]


In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_65_epoches.snn')
start = timeit.default_timer()
for i in range(65, 90):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1277s 39ms/step - loss: 1.4962e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_66_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_66_epoches.snn/assets


32932/32932 [==============================] - 1273s 39ms/step - loss: 1.4704e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_67_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_67_epoches.snn/assets


32932/32932 [==============================] - 1267s 38ms/step - loss: 1.4324e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_68_epoches.snn/assets


...

32932/32932 [==============================] - 1299s 39ms/step - loss: 1.1363e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_89_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_89_epoches.snn/assets


32932/32932 [==============================] - 1291s 39ms/step - loss: 1.1305e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_90_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_90_epoches.snn/assets


Time:  32181.181080901
[0.00014962266141083091, 0.00014704464410897344, 0.00014324235962703824, 0.00013830592797603458, 0.00013883912470191717, 0.00014359811029862612, 0.00014095754886511713, 0.00013703697186429054, 0.00013422605115920305, 0.00012997705198358744, 0.00013423188647720963, 0.0001330378872808069, 0.0001310816442128271, 0.00012571873958222568, 0.00012633160804398358, 0.00012849400809500366, 0.0001274347596336156, 0.00012529295054264367, 0.00012127200898248702, 0.0001251338835572824, 0.00011961520067416131, 0.00012110667739761993, 0.00011662135511869565, 0.00011363342491677031, 0.00011304792860755697]


In [ ]:
losses = []
SNN_model = keras.models.load_model('/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_90_epoches.snn')
start = timeit.default_timer()
for i in range(90, 100):
  train_data_gen = generator(trainset, num_negative, id2norm_add, dim)
  hist = SNN_model.fit(train_data_gen, verbose=1)
  losses.append(hist.history['loss'][0])
  SNN_model.save(filepath='_'.join(['/content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN', str(i+1), 'epoches.snn']))
stop = timeit.default_timer()
print('Time: ', stop - start) 
print(losses)

32932/32932 [==============================] - 1321s 40ms/step - loss: 1.1684e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_91_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_91_epoches.snn/assets


32932/32932 [==============================] - 1319s 40ms/step - loss: 1.1609e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_92_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_92_epoches.snn/assets


32932/32932 [==============================] - 1292s 39ms/step - loss: 1.1335e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_93_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_93_epoches.snn/assets


32932/32932 [==============================] - 1283s 39ms/step - loss: 1.1722e-04
INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_94_epoches.snn/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/Norm_add_based_recommendation/Full/model_1.2/ElementWise_model_1.4.4.2/SNN_94_epoches.snn/assets


    234/Unknown - 9s 40ms/step - loss: 9.3521e-05Buffered data was truncated after reaching the output size limit.